In [1]:
%cd ~/research

import gc
import os
import sys
from glob import glob

import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import yaml
from scipy import signal

sys.path.append("src")
from utility.activity_loader import get_data_dirs, load_individuals
from utility.video import Capture, Writer, concat_field_with_frame
from visualize.individual import write_field as ind_write_field
from visualize.keypoint import write_frame as kps_write_frame

/raid6/home/yokoyama/research


/home/yokoyama/research/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams["font.size"] = 24
plt.rcParams['xtick.direction'] = 'in'  # x axis in
plt.rcParams['ytick.direction'] = 'in'  # y axis in

In [3]:
room_num = "09"
surgery_num = "001"

In [4]:
cfg_path = "config/individual.yaml"
with open(cfg_path, "r") as f:
    cfg = yaml.safe_load(f)

In [5]:
field = cv2.imread("image/field.png")

In [6]:
data_dirs = get_data_dirs(room_num, surgery_num)
print(data_dirs)

['data/09/001/01', 'data/09/001/02', 'data/09/001/03', 'data/09/001/04', 'data/09/001/05', 'data/09/001/06', 'data/09/001/07', 'data/09/001/08', 'data/09/001/09', 'data/09/001/10', 'data/09/001/11', 'data/09/001/12', 'data/09/001/13', 'data/09/001/14', 'data/09/001/15', 'data/09/001/16']


In [7]:
individuals = []
for data_dir in tqdm(data_dirs[10:11]):
    json_path = os.path.join(data_dir, ".json", "individual.json")
    loaded = load_individuals(json_path, cfg)
    individuals += list(loaded.values())

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.18s/it]


In [ ]:
def calc_movements(individuals, th_length=900, move_size=5400, th_norm=10, pad_type="zero"):
    movements = {}
    positions = {}
    for ind in tqdm(individuals):
        pos_dict = ind.get_indicator_all("position")

        if len(pos_dict) <= th_length:
            continue
            
        move = []
        pos = []
        start_frame_num = pre_frame_num = list(pos_dict.keys())[0]
        for frame_num in pos_dict.keys():
            # calc diff
            pre_pos = np.array(pos_dict[pre_frame_num])
            crr_pos = np.array(pos_dict[frame_num])
            diff = crr_pos - pre_pos
            norm = np.linalg.norm(diff)
            
            if norm <= th_norm:
                frame_num_diff = frame_num - pre_frame_num
                for i in range(0, frame_num_diff - 1):
                    if pad_type == "zero":
                        # zero padding
                        move.append(0)
                        pos.append(pre_pos)
                    elif pad_type == "average":
                        # average padding
                        i += 1
                        move.append(norm / frame_num_diff * i)
                        pos.append(pre_pos + diff / frame_num_diff * i)
                move.append(norm)
                pos.append(crr_pos)

            pre_frame_num = frame_num

        movements[ind.id] = move
        positions[ind.id] = pos
    
    # crop movenemts for move_size
    ret_movements = {}
    ret_positions = {}
    for pid in tqdm(movements.keys()):
        count = 1
        for i in range(len(movements[pid]))[::move_size]:
            crop_move = movements[pid][i:i + move_size]
            crop_pos = positions[pid][i:i + move_size]
            
            # if len(crop_move) < move_size:
            #     # zero padding
            #     crop_move += np.full(move_size - len(crop_move), 0).tolist()
            #     crop_pos += [[0, 0] for _ in range(move_size - len(crop_pos))]
        
            ret_movements[f"{pid}_{count:03d}"] = crop_move
            ret_positions[f"{pid}_{count:03d}"] = crop_pos
            count += 1
        
    return ret_movements, ret_positions
    
movements, positions = calc_movements(individuals)